<div style="padding-top: 10px;  padding-bottom: 10px;">
  <img src="https://insightfactoryai.sharepoint.com/:i:/r/sites/insightfactory.ai/Shared%20Documents/E.%20Marketing/Company%20Logos%20and%20Style%20Guide/PNG/insightfactory.ai%20logo%20multiline%20reversed.png" alt='insightfactory.ai' width=150   style="display: block; margin: 0 auto" /> 
</div>

# Model Build

***Summary of process:*** This notebook is used to Build the ML model from the engineered features. This results in table with performance of the model and version of the resulting model. It is also preferred to Match your resulting model version with the used pipeline version in the resultant. 

***Input Tables:***  
- 

***Output Table:*** 
- model_config

Note: This is just the overview of the process, For details please review the notebook thoroughly

**Business Rules:** <br/>
\<Describe the Business Rules that are encapsulated in this Enrichment\>

**Dependencies:**<br/>
-

**Ownership:**<br/>
\<Indicate who owns this Enrichment ruleset\>


#### Modification Schedule

| Date | Who | Description |
| ---: | :--- | :--- |
| 2025-09-12 | Yifan Gu  | Initial version. |


#### Insight Factory Notebook Preparation

**(Do not modify/delete the following cell)**

In [0]:
%pip install pandas scikit-learn mlflow-skinny[databricks]
%pip install -U lightgbm
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/3.6 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/3.6 MB 1.5 MB/s eta 0:00:03
   ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/3.6 MB 537.1 kB/s eta 0:00:07
   ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/3.6 MB 537.1 kB/s eta 0:00:07
   ━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/3.6 MB 522.7 kB/s eta 0:00:07
   ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.2/3.6 MB 708.3 kB/s eta 0:00:05
   ━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.5/3.6 MB 2.0 MB/s eta 0:00:02
   ━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.9/3.6 MB 3.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 3.6/3.6 MB 11.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 3.6/3.6 MB 11.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 9.6 MB/s eta 0:00:00
Note: you may need to restart the kern

In [0]:
%run "/InsightFactory/Helpers/ML Build (Unity Catalog) Entry"

In [0]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, average_precision_score, roc_auc_score, f1_score,precision_recall_curve

from sklearn.feature_selection import RFECV
from lightgbm import LGBMClassifier
import numpy as np
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window
from sklearn.model_selection import TimeSeriesSplit


# Notebook Start

### Input Parameters

All Notebook Parameters (if any) are contained in the dictionary variable 'params'.  There are two ways to get the individual parameter from params, in both cases the parameter name is case-sensitive:

  1) Use dot-notation - refer to the example below.

      params = { "Name": "Test", "Values": { "Title": "Results", "Results": [ { "Definition": "Core Sample", "Outcome": "Prospective" }, { "Definition": "Follow-up", "Outcome": "For review" } ] } }

      params.Name produces 'Test'<br/>
      params.Values.Title produces 'Results'<br/>
      params.Values.Results[0] produces { "Definition": "Core Sample", "Outcome": "Prospective" }<br/>
      params.Values.Results[1].Definition produces 'Follow-up'

  2) Use the search_dictionary function as follows:  var1 = search_dictionary(params, "parameter-name").  

      There is an optional third parameter to this function: value_to_return_if_not_found -  this is the value to return if the particular parameter is not found in params.<br/>
      **Note** that value_to_return_if_not_found can take on any type (string, int, boolean, struct, ..) e.g search_dictionary(params, "IncorrectlyNamedParameter", False) will return the boolean False if "IncorrectlyNamedParameter" is not found in params.

**CAUTION:** There is another dictionary variable, 'config', that contains all of the configuration sent to this Notebook.  In most cases, you will have no use for 'config' but if you choose to use 'config' in this Notebook, note the following:
- Access the individual parameters within config by using the search_dictionary function e.g. search_dictionary(config, "ParameterName").  Dot-notation access **does not apply** to 'config'.
- Heed this **WARNING** - The individual parameter names within 'config' are subject to change outside of your control which may break your code.
<br/><br/>

### Enrichment Results

Add the code you need to perform your enrichment/extract in cell(s) below until the 'Notebook End' cell.

####Important: 
- Ensure that the result is stored in a PySpark dataframe as
    - 'df_result' e.g. df_result = ...  containing Model Name, Model Version, Performance metrics, Pipeline version (for feature log) and model
    - Ensure that your models are registered under ml_catalog. Always name your model as `f'{ml_catalog}.{delta_schema_name}.{model_name}'`

This will result in model and model config Table in your shared ml_catalog for easy sharing and inference across environments.
<br/><br/>

### Running this Notebook directly in Databricks

This Notebook can be run directly from your Databricks Workspace.  If the Notebook relies on Notebook Parameters, please read the following instructions:
1) Add this line of code to a cell at the top of your Notebook and run that cell.<br/>
   ```dbutils.widgets.text('ParametersJSON', '{ "ModelName":"name","ModelAlias":"alias","ModelVersion":"version","ModelSchema":"DatabaseName for Model", "NotebookParameters": { "param1": "value1", "param2": "value2" } }')```
2) This will add a Parameter to the Notebook.  Simply replace (or remove) the pre-canned parameters, 'param1', 'param2' and their values with your own.
3) When you have finished running this Notebook directly in Databricks, comment out the line of code you added or delete the cell entirely.    

In [0]:

# df = spark.table("`09ad024f-822f-48e4-9d9e-b5e03c1839a2`.feature_selection.total_training_table")

# keys = ["Tc_BaseCode", "Tc_BaseCode_Mapped", "Tc_SectionBreakStartKM", "Tc_r_date","p_key"]
# drop_cols = {"Wagon_RecordingDate",'Wagon_ICWVehicle'}

# numeric_cols = [f.name for f in df.schema.fields
#                 if isinstance(f.dataType, T.NumericType)
#                 and f.name not in drop_cols and f.name not in keys]

# other_keep = [c for c in df.columns if c not in keys and c not in drop_cols and c not in numeric_cols]
# avg_aggs   = [F.avg(c).alias(c) for c in numeric_cols]
# other_aggs = [F.first(c, ignorenulls=True).alias(c) for c in other_keep]

# df_day = (df.groupBy(*keys).agg(*(avg_aggs + other_aggs))
#             .withColumn("Tc_r_date", F.to_date("Tc_r_date")))

# grp = ["Tc_BaseCode", "Tc_BaseCode_Mapped", "Tc_SectionBreakStartKM"]
# w_order = Window.partitionBy(*grp).orderBy(F.col("Tc_r_date").cast("timestamp"))

# def rolling_feats(df_in, cols, windows=[7,14,30]):
#     df_out = df_in
#     for c in cols:
#         df_out = df_out.withColumn(f"{c}_lag1", F.lag(c, 1).over(w_order))

#         for W in windows:
#             w = w_order.rowsBetween(-W, -1)
#             df_out = (df_out
#                 .withColumn(f"{c}_mean_{W}", F.avg(F.col(f"{c}_lag1")).over(w))
#                 .withColumn(f"{c}_std_{W}",  F.stddev(F.col(f"{c}_lag1")).over(w))
#                 .withColumn(f"{c}_max_{W}",  F.max(F.col(f"{c}_lag1")).over(w))
#                 .withColumn(f"{c}_min_{W}",  F.min(F.col(f"{c}_lag1")).over(w))
#             )

#         df_out = df_out.withColumn(
#             f"{c}_diff1",
#             F.col(f"{c}_lag1") - F.lag(f"{c}_lag1", 1).over(w_order)
#         )

#         if 7 in windows and 30 in windows:
#             df_out = df_out.withColumn(
#                 f"{c}_sma7_div_sma30",
#                 F.when(F.col(f"{c}_mean_30").isNull() | (F.col(f"{c}_mean_30")==0), F.lit(None).cast("double"))
#                  .otherwise(F.col(f"{c}_mean_7")/F.col(f"{c}_mean_30"))
#             )
#     return df_out

# sensor_cols = [
#  'Wagon_Twist14m','Wagon_BounceFrt','Wagon_BounceRr','Wagon_BodyRockFrt','Wagon_BodyRockRr',
#  'Wagon_LP1','Wagon_LP2','Wagon_LP3','Wagon_LP4','Wagon_Speed','Wagon_BrakeCylinder',
#  'Wagon_IntrainForce','Wagon_Acc1','Wagon_Acc2','Wagon_Acc3','Wagon_Acc4','Wagon_Twist2m',
#  'Wagon_Acc1_RMS','Wagon_Acc2_RMS','Wagon_Acc3_RMS','Wagon_Acc4_RMS','Wagon_Rail_Pro_L',
#  'Wagon_Rail_Pro_R','Wagon_SND','Wagon_VACC','Wagon_VACC_L','Wagon_VACC_R','Wagon_Curvature',
#  'Wagon_Track_Offset','Wagon_SND_L','Wagon_SND_R','w_row_count','Tng_Tonnage'
# ]

# df_time = rolling_feats(df_day, sensor_cols, windows=[7,14,30])
 
# df_time = (df_time
#     .withColumn("dow", F.dayofweek("Tc_r_date"))   # 1..7
#     .withColumn("month", F.month("Tc_r_date"))
#     .withColumn("sin_dow",  F.sin(2*3.1415926*(F.col("dow")-1)/7.0))
#     .withColumn("cos_dow",  F.cos(2*3.1415926*(F.col("dow")-1)/7.0))
#     .withColumn("sin_mon",  F.sin(2*3.1415926*(F.col("month")-1)/12.0))
#     .withColumn("cos_mon",  F.cos(2*3.1415926*(F.col("month")-1)/12.0))
# )

# # w_hist = w_order.rowsBetween(Window.unboundedPreceding, -1)
# # df_time = (df_time
# #     .withColumn("last_break_date_hist", F.last("Tc_break_date", ignorenulls=True).over(w_hist))
# #     # .withColumn("last_fail_date_hist",  F.last("Tc_last_fail_if_available_otherwise_null", ignorenulls=True).over(w_hist))
# #     .withColumn("days_since_last_break", F.when(F.col("last_break_date_hist").isNull(), None)
# #                 .otherwise(F.datediff(F.col("Tc_r_date"), F.to_date("last_break_date_hist"))))
# #     #.withColumn("days_since_last_fail", F.when(F.col("last_fail_date_hist").isNull(), None)
# #     #            .otherwise(F.datediff(F.col("Tc_r_date"), F.to_date("last_fail_date_hist"))))
# # )

# target_col = "Tc_target"
# aux_cols = ["Tc_BaseCode","Tc_BaseCode_Mapped","Tc_SectionBreakStartKM","p_key","Tc_r_date","Tc_rul","Tc_break_date",                "Tc_last_fail_if_available_otherwise_null"]
# #            "Tc_last_fail_if_available_otherwise_null","last_break_date_hist"]

# time_feat_cols = [c for c in df_time.columns
#                   if any(s in c for s in ["_mean_","_std_","_max_","_min_","_diff1","_sma7_div_sma30",
#                                           "sin_","cos_"])]

# use_cols = aux_cols + sensor_cols + time_feat_cols + [target_col]
# df_feat = df_time.select(*[c for c in use_cols if c in df_time.columns])

In [0]:
# pdf = df_feat.select(*use_cols).toPandas()
# pdf["Tc_r_date"] = pd.to_datetime(pdf["Tc_r_date"])

# pdf = pdf.sort_values(
#     ["Tc_r_date","Tc_BaseCode","Tc_BaseCode_Mapped","Tc_SectionBreakStartKM","p_key"],
#     kind="mergesort"   
# ).reset_index(drop=True)

# import numpy as np
# X_all = (pdf[sensor_cols + time_feat_cols]
#          .apply(pd.to_numeric, errors="coerce")
#          .replace([np.inf, -np.inf], np.nan)
#          .astype("double"))
# y_all = pdf[target_col].astype(int).to_numpy()

# uniq_dates = np.sort(pdf["Tc_r_date"].unique())
# cutoff_date = uniq_dates[int(len(uniq_dates) * 0.8) - 1]   

# train_mask = pdf["Tc_r_date"] <= cutoff_date
# test_mask  = pdf["Tc_r_date"]  > cutoff_date

# X_tr, y_tr = X_all[train_mask], y_all[train_mask]
# X_te, y_te = X_all[test_mask],  y_all[test_mask]

# print("train:", X_tr.shape, "test:", X_te.shape)

# pos = int(y_tr.sum())
# neg = int(len(y_tr) - pos)
# spw = max(1.0, neg / max(1, pos))
# print("train pos/neg:", pos, "/", neg, "=> spw =", round(spw,2))


train: (41867, 499) test: (3837, 499)
train pos/neg: 4683 / 37184 => spw = 7.94


In [0]:
df = spark.table("`09ad024f-822f-48e4-9d9e-b5e03c1839a2`.feature_selection.preprocess_training_table")

# Grouping columns
keys = ["Tc_BaseCode", "Tc_BaseCode_Mapped", "Tc_SectionBreakStartKM", "Tc_r_date"]

# Exclude columns
drop_cols = {"Wagon_RecordingDate",'Wagon_ICWVehicle'}

# Averaging only numeric columns
numeric_cols = [
    f.name
    for f in df.schema.fields
    if isinstance(f.dataType, T.NumericType)          
    and f.name not in drop_cols
    and f.name not in keys
]
print(numeric_cols)


# Constructing aggregate expressions
avg_aggs = [F.avg(F.col(c)).alias(c) for c in numeric_cols]

# Take the first non-empty column that is not numeric and not excluded
other_keep = [
    c for c in df.columns
    if c not in keys and c not in drop_cols and c not in numeric_cols
]
other_aggs = [F.first(F.col(c), ignorenulls=True).alias(c) for c in other_keep]

# Grouping + Aggregation
df_merged = df.groupBy(*keys).agg(*(avg_aggs + other_aggs))

# Fill all the averaged columns with 0 again
df_merged = df_merged.fillna(0, subset=numeric_cols)

df_merged.display()   

In [0]:
target_col = "Tc_target"
aux_cols   = ["Tc_BaseCode", "Tc_r_date",'Tc_rul','Tc_break_date', 'Tc_last_fail_if_available_otherwise_null','Tc_BaseCode_Mapped', 'Tc_SectionBreakStartKM',] 

candidate_cols = [
     'Wagon_Twist14m', 'Wagon_BounceFrt', 'Wagon_BounceRr', 'Wagon_BodyRockFrt', 'Wagon_BodyRockRr', 'Wagon_LP1', 'Wagon_LP2', 'Wagon_LP3', 'Wagon_LP4', 'Wagon_Speed', 'Wagon_BrakeCylinder', 'Wagon_IntrainForce', 'Wagon_Acc1', 'Wagon_Acc2', 'Wagon_Acc3', 'Wagon_Acc4', 'Wagon_Twist2m', 'Wagon_Acc1_RMS', 'Wagon_Acc2_RMS', 'Wagon_Acc3_RMS', 'Wagon_Acc4_RMS', 'Wagon_Rail_Pro_L', 'Wagon_Rail_Pro_R', 'Wagon_SND', 'Wagon_VACC', 'Wagon_VACC_L', 'Wagon_VACC_R', 'Wagon_Curvature', 'Wagon_Track_Offset', 'Wagon_SND_L', 'Wagon_SND_R', 'w_row_count', 'Tng_Tonnage'
]

use_cols = candidate_cols + [target_col] + aux_cols
pdf = df_merged.select(*use_cols).toPandas()


import numpy as np
import pandas as pd

X_all = (pdf[candidate_cols]
         .apply(pd.to_numeric, errors="coerce")
         .replace([np.inf, -np.inf], np.nan)
         .fillna(0.0)
         .astype("double"))
y_all = pdf[target_col].astype(int).values

# Sorting by time segmentation
pdf["_order"] = pd.to_datetime(pdf["Tc_r_date"])
order = np.argsort(pdf["_order"].values)
X_all = X_all.iloc[order].reset_index(drop=True)
y_all = y_all[order]
pdf = pdf.iloc[order].reset_index(drop=True) 
# groups = pdf["Tc_BaseCode"].values[order]   

split_ratio = 0.8
split_idx = int(len(X_all) * split_ratio)
train_mask = np.arange(len(X_all)) < split_idx
test_mask  = ~train_mask

X_tr, y_tr = X_all.loc[train_mask].reset_index(drop=True), y_all[train_mask]
X_te, y_te = X_all.loc[test_mask].reset_index(drop=True), y_all[test_mask]

print("train size:", X_tr.shape, "test size:", X_te.shape)
      
pos = (y_tr == 1).sum()
neg = (y_tr == 0).sum()
spw = max(1.0, neg / max(1,pos))
print(f"train pos/neg: {pos}/{neg} = {spw:.2f}")

In [0]:

selected_feats = [
    'Wagon_Acc1', 'Wagon_Acc1_RMS', 'Wagon_Acc2', 'Wagon_Acc2_RMS',
    'Wagon_Acc3', 'Wagon_Acc3_RMS', 'Wagon_Acc4', 'Wagon_Acc4_RMS'
]
tscv = TimeSeriesSplit(n_splits=5)
X_sel = X_tr[selected_feats]
#X_sel = X_tr
pr_scores, roc_scores, f1_scores, acc_scores = [], [], [], []

lgb = LGBMClassifier(
    objective="binary",
    n_estimators=2000,
    learning_rate=0.03,
    num_leaves=31,
    min_child_samples=30,
    subsample=0.8, colsample_bytree=0.8,
    reg_lambda=1.0,
    scale_pos_weight=spw,
    feature_pre_filter=False,
    random_state=42, n_jobs=-1
)


ths = []
for tr_idx, va_idx in tscv.split(X_sel):            
    X_tr_thr = X_sel.iloc[tr_idx]
    X_va_thr = X_sel.iloc[va_idx]

    y_tr_thr = y_tr[tr_idx] 
    y_va_thr = y_tr[va_idx] 

    mdl = LGBMClassifier(**lgb.get_params())
    mdl.fit(
        X_tr_thr, y_tr_thr,
        eval_set=[(X_va_thr, y_va_thr)],
        eval_metric="aucpr",
        callbacks=[__import__("lightgbm").early_stopping(stopping_rounds=50, verbose=False)]
    )

    p_va = mdl.predict_proba(X_va_thr)[:, 1]
    prec, reca, thr = precision_recall_curve(y_va_thr, p_va)  
    f1 = 2 * prec * reca / (prec + reca + 1e-9)

    ths.append(thr[np.nanargmax(f1[:-1])])   

best_thr = float(np.median(ths))
print(f"Chosen threshold from CV (median): {best_thr:.6f}")
#best_thr = 0.999448

# Retrain with the entire training set
final_model = LGBMClassifier(**lgb.get_params())
final_model.fit(X_sel, y_tr)

# Do a final evaluation on the test set
X_sel_te = X_te[selected_feats]
#X_sel_te = X_te
proba_te = final_model.predict_proba(X_sel_te)[:, 1]
y_pred_te = (proba_te >= best_thr).astype(int)

pr_auc = float(average_precision_score(y_te, proba_te))
f1 = float(f1_score(y_te, y_pred_te))
acc = float(accuracy_score(y_te, y_pred_te))

print("\n=== TEST  ===")
print("PR-AUC:", pr_auc)
print("F1    :", f1)
print("ACC   :", acc)

[LightGBM] [Info] Number of positive: 478, number of negative: 6504
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.114892 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 119964
[LightGBM] [Info] Number of data points in the train set: 6982, number of used features: 499
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.068462 -> initscore=-2.610562
[LightGBM] [Info] Start training from score -2.610562
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 1362, number of negative: 12597
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the 

In [0]:
# print(f"Chosen threshold from CV (median): {best_thr:.6f}")
# n_pos_pred = int(np.sum(y_pred_te == 1))
# print( n_pos_pred)

Chosen threshold from CV (median): 0.999448
预测为1的个数: 944


In [0]:
import json, time
import mlflow, mlflow.lightgbm
from mlflow.models.signature import infer_signature
from mlflow.tracking import MlflowClient
from sklearn.metrics import average_precision_score, roc_auc_score, f1_score, accuracy_score
from pyspark.sql import Row

registered_name = f"{ml_catalog}.{model_schema_name}.{model_name}"

with mlflow.start_run() as run:
    mlflow.log_params({
        "n_estimators": 2000,
        "learning_rate": 0.03,
        "num_leaves": 31,
        "min_child_samples": 30,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "reg_lambda": 1.0,
        "scale_pos_weight": float(spw),
        "random_state": 42,
        "selected_feat_cnt": len(selected_feats),
    })

    mlflow.log_metrics({
        "PR_AUC": pr_auc,
        "F1    ": f1,
        "ACC   ": acc
    })

    mlflow.log_dict(
        {"selected_feats": selected_feats, "threshold": float(best_thr)},
        artifact_file="rfe_lightgbm.json"
    )

    ex_X = X_sel_te.head(5)
    print(ex_X.columns)
    sign = infer_signature(model_input=ex_X, model_output=final_model.predict_proba(ex_X)[:, 1])

    mlflow.lightgbm.log_model(
        final_model,
        artifact_path="model",                    
        signature=sign,
        input_example=ex_X.iloc[:1],
        registered_model_name=registered_name      
    )

    run_id = run.info.run_id
    print(f"[MLflow] run_id = {run_id}")

client = MlflowClient()
model_version = None
for _ in range(10):
    mvs = [m for m in client.search_model_versions(f"name='{registered_name}'") if m.run_id == run_id]
    if mvs:
        model_version = int(mvs[0].version)
        break
    time.sleep(1)

print(f"[Registry] {registered_name} version = {model_version}")

df_result = spark.createDataFrame([
    Row(
        ModelName      = registered_name,                 
        ModelVersion   = model_version,                
        run_id          = run_id,
        test_pr_auc     = pr_auc,
        test_f1         = f1,
        test_acc        = acc,
        threshold       = float(best_thr),
        selected_feats  = json.dumps(selected_feats),      
    )
])


In [0]:
# ################# Update your output data for the model configuration here #################
# import pandas as pd
# df_result=spark.createDataFrame(pd.DataFrame(
#     data=[[]],
#     columns=['ModelName','ModelVersion','ModelMetrics','PipelineVersion']
# ))


# Notebook End

**(Do not modify/delete the following cell)**

####Important: 
1) Ensure that the result is stored in a PySpark dataframe as
  - 'df_result' e.g. df_result = ...  containing Model Name, Model Version, Performance metrics, Pipeline version (for feature log)
2) Ensure that your models are registered under ml_catalog. Always name your model as `f'{ml_catalog}.{delta_schema_name}.{model_name}'`

This will result in model and model config Table in your shared ml_catalog for easy sharing and inference across environments.

In [0]:
%run "/InsightFactory/Helpers/ML Build (Unity Catalog) Exit"

# Testing or Debugging Zone